In [11]:
!pip install chardet

   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   --------------------------------------- 199.4/199.4 kB 11.8 MB/s eta 0:00:00


In [26]:
!pip install nltk textblob

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 48.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   --------------------------------------- 626.3/626.3 kB 41.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   --------------------------------------- 269.0/269.0 kB 16.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ---------------------------------------- 97.9/97.9 kB ? eta 0:00:00


In [33]:
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import nltk
import chardet

# nltk 데이터 다운로드 (첫 실행 시 필요)
nltk.download('vader_lexicon')

# nltk 데이터 다운로드 (첫 실행 시 필요)
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [12]:


# Detect encoding
with open('../../../../datasets/reviews.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Read file with detected encoding
encoding = result['encoding']
print(f"Detected encoding: {encoding}")
reviews = pd.read_csv('../../../../datasets/reviews.csv', encoding=encoding)

Detected encoding: MacRoman


In [15]:

calendar = pd.read_csv('../../../../datasets/calendar.csv', encoding='cp949')
listings = pd.read_csv('../../../../datasets/listings.csv', encoding=encoding)


C:\Users\ciw96\AppData\Local\Temp\ipykernel_37152\1961811894.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv('../../../../datasets/calendar.csv', encoding='cp949')


In [19]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [23]:
len(listings.groupby('id'))

7707

In [24]:
len(listings)

7707

In [25]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2384,25218143,2015-01-09,14385014,Ivan,it's a wonderful trip experience. I didn't exc...
1,2384,28475392,2015-03-24,16241178,Namhaitou,This is my first trip using Airbnb. I was a li...
2,2384,30974202,2015-04-30,26247321,Cristina,S√≥lo puedo decir cosas buenas de Rebecca. La ...
3,2384,31363208,2015-05-04,31293837,SuJung,Rebecca was an absolutely wonderful host.\r<br...
4,2384,31820011,2015-05-10,2873370,Krishanu,Rebecca really tried to make it feel like home...
...,...,...,...,...,...,...
389201,1110190322403262176,1118246647795659114,2024-03-22,2536962,Cathey,If you want a hassle free and seamless experie...
389202,1110190322403262176,1118961435549597868,2024-03-23,127697383,Jerrod,I absolutely loved the place beautiful view of...
389203,1110851028892552028,1116027707663736454,2024-03-19,298534,Abraham,We and our kitties loved staying at Bella‚Äôs ...
389204,1110930497116978967,1114575547663090186,2024-03-17,544856301,Bianca,A wonderful place to stay! Very nice and safe ...


In [35]:


# 감성 분석 함수 정의
def analyze_sentiment_vader(comment):
    sid = SentimentIntensityAnalyzer()
    scores = sid.polarity_scores(comment)
    return scores

def analyze_sentiment_textblob(comment):
    analysis = TextBlob(comment)
    return analysis.sentiment.polarity

# VADER 감성 분석 결과 추가
reviews['vader_sentiment'] = reviews['comments'].apply(lambda x: analyze_sentiment_vader(str(x)))

# TextBlob 감성 분석 결과 추가
reviews['textblob_sentiment'] = reviews['comments'].apply(lambda x: analyze_sentiment_textblob(str(x)))

# VADER 점수 세부 항목 분리
reviews['vader_neg'] = reviews['vader_sentiment'].apply(lambda x: x['neg'])  # 부정적인 감성의 비율
reviews['vader_neu'] = reviews['vader_sentiment'].apply(lambda x: x['neu'])  # 중립적인 감성의 비율
reviews['vader_pos'] = reviews['vader_sentiment'].apply(lambda x: x['pos'])  # 긍정적인 감성의 비율
reviews['vader_compound'] = reviews['vader_sentiment'].apply(lambda x: x['compound'])  # 전체적인 감성 점수

# 결과 출력
print(reviews[['comments', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound', 'textblob_sentiment']].head())


                                            comments  vader_neg  vader_neu  \
0  it's a wonderful trip experience. I didn't exc...      0.000      0.690   
1  This is my first trip using Airbnb. I was a li...      0.037      0.776   
2  Sólo puedo decir cosas buenas de Rebecca. La h...      0.000      1.000   
3  Rebecca was an absolutely wonderful host.\r<br...      0.038      0.847   
4  Rebecca really tried to make it feel like home...      0.020      0.839   

   vader_pos  vader_compound  textblob_sentiment  
0      0.310          0.9489            0.460000  
1      0.187          0.9519            0.266859  
2      0.000          0.0000            0.000000  
3      0.116          0.9669            0.228750  
4      0.141          0.9809            0.215431  


neg (Negative): 부정적인 감성의 비율 (0에서 1 사이)

neu (Neutral): 중립적인 감성의 비율 (0에서 1 사이)

pos (Positive): 긍정적인 감성의 비율 (0에서 1 사이)

compound: 전체적인 감성 점수 (-1에서 1 사이)

In [28]:
reviews[['comments', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound', 'textblob_sentiment']].head()

,comments,vader_neg,vader_neu,vader_pos,vader_compound,textblob_sentiment
0,it's a wonderful trip experience. I didn't exc...,0.000,0.690,0.310,0.9489,0.460000
1,This is my first trip using Airbnb. I was a li...,0.037,0.776,0.187,0.9519,0.266859
2,S√≥lo puedo decir cosas buenas de Rebecca. La ...,0.000,1.000,0.000,0.0000,0.000000
3,Rebecca was an absolutely wonderful host.\r<br...,0.038,0.847,0.116,0.9669,0.228750
4,Rebecca really tried to make it feel like home...,0.020,0.839,0.141,0.9809,0.215431


In [50]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

# nltk 데이터 다운로드 (첫 실행 시 필요)
nltk.download('stopwords')

# 긍정적 리뷰와 부정적 리뷰로 분류
positive_reviews = reviews[reviews['vader_pos'] > 0.5]['comments'].dropna().astype(str)
negative_reviews = reviews[reviews['vader_neg'] > 0.5]['comments'].dropna().astype(str)

# 영어, 스페인어, 독일어 불용어 목록 결합 및 리스트로 변환
stop_words = list(set(stopwords.words('english')) | set(stopwords.words('spanish')) | set(stopwords.words('german')))

# 추가로 제외할 단어 목록
additional_stop_words = ['place', 'stay', 'chicago', 'airbnb', 'would', 'us']

# 추가적인 불용어 목록을 결합
stop_words.extend(additional_stop_words)

# TF-IDF 벡터라이저 초기화 (결합된 불용어 목록 사용)
tfidf_vectorizer = TfidfVectorizer(max_features=100, stop_words=stop_words)

# 긍정적 리뷰에 대한 TF-IDF 행렬 계산
tfidf_matrix_positive = tfidf_vectorizer.fit_transform(positive_reviews)
feature_names_positive = tfidf_vectorizer.get_feature_names_out()
tfidf_scores_positive = tfidf_matrix_positive.sum(axis=0).A1
tfidf_df_positive = pd.DataFrame({'word': feature_names_positive, 'tfidf_score': tfidf_scores_positive})
top_tfidf_positive = tfidf_df_positive.nlargest(20, 'tfidf_score')  # 상위 20개 단어

# 부정적 리뷰에 대한 TF-IDF 행렬 계산 (불용어 목록 조정)
try:
    tfidf_matrix_negative = tfidf_vectorizer.fit_transform(negative_reviews)
    feature_names_negative = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores_negative = tfidf_matrix_negative.sum(axis=0).A1
    tfidf_df_negative = pd.DataFrame({'word': feature_names_negative, 'tfidf_score': tfidf_scores_negative})
    top_tfidf_negative = tfidf_df_negative.nlargest(20, 'tfidf_score')  # 상위 20개 단어

    # 결과 출력
    print("긍정적인 리뷰에서 상위 20개 단어:")
    print(top_tfidf_positive)
    print("\n부정적인 리뷰에서 상위 20개 단어:")
    print(top_tfidf_negative)
except ValueError as e:
    print(f"TF-IDF 계산 중 오류 발생: {e}")
    print("부정적 리뷰의 예시:")
    print(negative_reviews.head(10))  # 부정적 리뷰 예시 출력


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ciw96\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


긍정적인 리뷰에서 상위 20개 단어:
            word   tfidf_score
45         great  17524.249445
58      location   8603.013131
19         clean   6009.835195
66          nice   5658.699759
50          host   4827.444589
44          good   3782.576959
22   comfortable   3329.535812
75     recommend   3167.458823
4        amazing   2786.357109
88         super   2741.192146
30    definitely   2739.778239
70       perfect   2683.794262
10     beautiful   2633.794407
32          easy   2497.231951
81         space   2477.426282
99     wonderful   2302.613353
77    responsive   2174.255572
6      apartment   2161.691001
8        awesome   2105.381993
65  neighborhood   1984.021953

부정적인 리뷰에서 상위 20개 단어:
          word  tfidf_score
11  complaints    30.605023
20       great     9.786606
9      comment     8.000000
52    problems     7.524254
5          bad     7.248404
72    terrible     7.149223
10    comments     6.632014
31        nice     5.135513
58   recommend     5.066717
64        rude     4.95127

In [51]:
top_tfidf_positive

,word,tfidf_score
45,great,17524.249445
58,location,8603.013131
19,clean,6009.835195
66,nice,5658.699759
50,host,4827.444589
44,good,3782.576959
22,comfortable,3329.535812
75,recommend,3167.458823
4,amazing,2786.357109
88,super,2741.192146


In [52]:
top_tfidf_negative

,word,tfidf_score
11,complaints,30.605023
20,great,9.786606
9,comment,8.000000
52,problems,7.524254
5,bad,7.248404
72,terrible,7.149223
10,comments,6.632014
31,nice,5.135513
58,recommend,5.066717
64,rude,4.951277


In [48]:
# 'br' 문자열을 포함하는 리뷰 필터링
br_reviews = reviews[reviews['comments'].str.contains('br', na=False)]

# 필터링된 리뷰 출력
print(br_reviews['comments'])

0         it's a wonderful trip experience. I didn't exc...
3         Rebecca was an absolutely wonderful host.\r<br...
4         Rebecca really tried to make it feel like home...
5         Hi. i just return from Chicago and want to sha...
6         Rebecca is a very charming and  nice personn, ...
                                ...                        
389180    I can not say enough positive things about my ...
389185    The host was wonderful no doubt, they were ver...
389186    It was okay somethings were broken and it was ...
389201    If you want a hassle free and seamless experie...
389203    We and our kitties loved staying at Bella’s pl...
Name: comments, Length: 61222, dtype: object


다른방식

In [69]:
!pip install --upgrade scipy gensim

In [76]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
import networkx as nx
from nltk.corpus import stopwords
import nltk

# 1. Term Frequency (단어 빈도)
def term_frequency(reviews, stop_words):
    words = [word for word in ' '.join(reviews).split() if word.lower() not in stop_words]
    word_counts = Counter(words)
    return [word for word, _ in word_counts.most_common(20)]

most_common_positive = term_frequency(positive_reviews, stop_words)
most_common_negative = term_frequency(negative_reviews, stop_words)

# 2. Latent Dirichlet Allocation (LDA)
def lda_keywords(reviews, stop_words):
    count_vectorizer = CountVectorizer(stop_words=stop_words)
    count_data = count_vectorizer.fit_transform(reviews)
    lda = LatentDirichletAllocation(n_components=1, random_state=42)
    lda.fit(count_data)
    topic = lda.components_
    feature_names = count_vectorizer.get_feature_names_out()
    top_words = [feature_names[i] for i in topic[0].argsort()[-20:]]
    return top_words

top_words_positive_lda = lda_keywords(positive_reviews, stop_words)
top_words_negative_lda = lda_keywords(negative_reviews, stop_words)

# 3. TextRank
def textrank_keywords(texts, stop_words, top_n=20):
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    
    # 단어 간의 유사성 그래프 생성
    similarity_graph = (X.T * X)
    similarity_graph.setdiag(0)
    similarity_graph = similarity_graph.toarray()
    
    # 네트워크 그래프 생성
    nx_graph = nx.from_numpy_array(similarity_graph)
    scores = nx.pagerank(nx_graph)
    
    # 점수 기준 상위 단어 추출
    ranked_words = sorted(((scores[i], s) for i, s in enumerate(feature_names)), reverse=True)
    top_keywords = [word for _, word in ranked_words[:top_n]]
    
    return top_keywords

positive_keywords_textrank = textrank_keywords(positive_reviews, stop_words)
negative_keywords_textrank = textrank_keywords(negative_reviews, stop_words)

# 4. TF-IDF
def tfidf_keywords(reviews, stop_words, top_n=20):
    vectorizer = TfidfVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(reviews)
    feature_names = vectorizer.get_feature_names_out()
    scores = X.sum(axis=0).A1
    sorted_indices = scores.argsort()[::-1]
    top_keywords = [feature_names[i] for i in sorted_indices[:top_n]]
    return top_keywords

top_words_positive_tfidf = tfidf_keywords(positive_reviews, stop_words)
top_words_negative_tfidf = tfidf_keywords(negative_reviews, stop_words)

# 겹치는 단어 찾기
def find_common_words(lists):
    common_words = set(lists[0])
    for lst in lists[1:]:
        common_words.intersection_update(lst)
    return list(common_words)

positive_lists = [most_common_positive, top_words_positive_lda, positive_keywords_textrank, top_words_positive_tfidf]
negative_lists = [most_common_negative, top_words_negative_lda, negative_keywords_textrank, top_words_negative_tfidf]

common_positive_words = find_common_words(positive_lists)
common_negative_words = find_common_words(negative_lists)

print("긍정적인 리뷰에서 공통적으로 상위에 있는 단어:")
print(common_positive_words)

print("\n부정적인 리뷰에서 공통적으로 상위에 있는 단어:")
print(common_negative_words)

긍정적인 리뷰에서 공통적으로 상위에 있는 단어:
['good', 'nice', 'location', 'host', 'definitely', 'easy', 'super', 'comfortable', 'recommend', 'clean', 'great']

부정적인 리뷰에서 공통적으로 상위에 있는 단어:
['nice', 'experience', 'bad', 'rude', 'problems', 'complaints', 'great']
